<a href="https://colab.research.google.com/github/elminhodev/Simulador-de-Automatos/blob/main/automotos2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

In [ ]:
import csv

In [ ]:
import time

In [ ]:
def carregar_automato(nome_arquivo):
    with open(nome_arquivo, 'r') as f:
        data = json.load(f)
    inicial = data["initial"]
    finais = set(data["final"])
    transicoes = {}
    for t in data["transitions"]:
        origem = t["from"]
        simbolo = t["read"]  # pode ser None
        destino = t["to"]
        transicoes.setdefault(origem, {}).setdefault(simbolo, []).append(destino)
    return inicial, finais, transicoes

def epsilon_closure(estados, transicoes):
    pilha = list(estados)
    closure = set(estados)
    while pilha:
        estado = pilha.pop()
        for prox in transicoes.get(estado, {}).get(None, []):
            if prox not in closure:
                closure.add(prox)
                pilha.append(prox)
    return closure

def simular_automato(inicial, finais, transicoes, palavra):
    estados_atuais = epsilon_closure({inicial}, transicoes)
    for simbolo in palavra:
        proximos_estados = set()
        for estado in estados_atuais:
            for destino in transicoes.get(estado, {}).get(simbolo, []):
                proximos_estados.update(epsilon_closure({destino}, transicoes))
        estados_atuais = proximos_estados
    return any(estado in finais for estado in estados_atuais)

def executar_simulador():
    inicial, finais, transicoes = carregar_automato("ex3_naodeterministico.aut")
    with open("ex3_naodeterministico_input.in", 'r') as f_in, open("ex3_naodeterministico_input.out", 'w') as f_out:
        leitor = csv.reader(f_in, delimiter=';')
        for linha in leitor:
            palavra, esperado = linha[0], int(linha[1])
            inicio = time.time()
            obtido = int(simular_automato(inicial, finais, transicoes, palavra))
            tempo = round(time.time() - inicio, 3)
            f_out.write(f"{palavra};{esperado};{obtido};{tempo}\n")

executar_simulador()

with open("ex3_naodeterministico_input.out", "r") as f:
    print(f.read())


b;1;1;0.0
aab;1;1;0.0
ab;1;1;0.0
aaaab;1;1;0.0
a;0;0;0.0
;0;0;0.0
aa;0;0;0.0
bb;0;0;0.0

